Notebook elaborado para estudar algumas características básicas da análise de dados GPR adquiridos em modo de afastamento constante.
Criado para a disciplina Geofísica II da Universidade Federal de Uberlândia.
O uso e a reprodução são livres para fins educacionais, pede-se apenas a citação da fonte.

[![License: GPL v3](https://img.shields.io/badge/License-GPLv3-blue.svg)](https://www.gnu.org/licenses/gpl-3.0)

Esta demonstração utiliza dados adquiridos sobre o Sítio Controlado de Geofísica Rasa do Instituto de Geofísica, Astronomia e Ciências Atmosféricas da Universidade de São Paulo (BORGES, 2007) e adquiridos sobre sedimentos costeiros da região de Jaguaruna em Santa Catarina (FORNARI, 2010).


Prof. Dr. Emerson Rodrigo Almeida<br>
Universidade Federal de Uberlândia<br>
Instituto de Geografia - Curso de Geologia<br>
emerson.almeida@ufu.br<br>

<p>&nbsp;</p>
Última modificação: 23/09/2022

---
### Referências

BORGES, W. R. **Caracterização geofísica de alvos rasos com aplicações no planejamento urbano e meio ambiente: estudo sobre o sítio controlado do IAG/USP**. Tese de Doutorado. São Paulo: Instituto de Astronomia, Geofísica e Ciências Atmosféricas, Universidade de São Paulo, 2010.

FORNARI, M. **Evolução sedimentar holocênica da retrobarreira na região de Jaguaruna-Laguna, Santa Catarina, Brasil**. Tese de Doutorado. São Paulo: Instituto de Geociências, Universidade de São Paulo, 2010.

---

<p>&nbsp;</p>

# Instruções iniciais

Neste notebook você irá trabalhar as ferramentas de ganho e de filtragem, fundamentais tanto no processamento de dados GPR quanto no processmamento de dados sísmicos. Para isto leia atentamente as instruções apresentadas antes de cada célula e execute-as uma a uma, para acompanhar o processo corretamente.

Não se preocupe em compreender o código em si, não é obrigatório que você conheça a programação para obter e interpretar os resultados da demonstração. Mesmo assim, sinta-se livre caso queira perguntar qualquer coisa para aprender mais sobre o código e sua elaboração.

<p>&nbsp;</p>

### Passo 01 - Preparação

Você pode executar o código na célula clicando sobre ela e em seguida clicando no botão **RUN** na barra do Jupyter Notebook ou pressionando as teclas **SHIFT+ENTER** no seu teclado.

<p>&nbsp;</p>

**Importante!** Nem todas as células irão retornar algum tipo de mensagem ou figura após a execução. Isto é normal e não significa que a execução tenha falhado. Perceba o símbolo **In [   ]:** próximo ao canto superior esquerdo de cada célula. Ao iniciar a execução de uma delas você verá um asterisco ($*$) aparecer dentro dos colchetes. Você saberá que a execução ocorreu sem problemas quando o $*$ dentro dos colchetes for substituído por um número e nenhuma mensagem de erro aparecer abaixo da célula executada./

In [ ]:
import modulos.gpr as gpr

# Passo 02 - Importação dos dados

Neste passo você irá carregar os dados brutos que serão trabalhados. Há três opções de dados para você utilizar:

- **270_cabos.dat**: dados adquiridos com uma antena de 270 MHz com distância de 2 cm entre traços sobre um conjunto de cabos elétricos;
- **270_tubos.dat**: dados adquiridos com uma antena de 270 MHz com distância de 2 cm entre traços sobre um conjunto de dutos metálicos;
- **200_SC.dat**: dados adquiridos com uma antena de 200 MHz com distância de 10 cm entre traços sobre sedimentos costeiros.

O arquivo de dado pode ser selecionado escolhendo-se um dos nomes de arquivo em negrito e inserindo-o (incluindo a extensão '.dat') na primeira linha da célula abaixo. É possível contaminar o dado com ruído em uma determinada faixa de frequências atribuindo-se uma frequência de referência (em Hz) para a variável $ruido$ na segunda linha, mas você não fará isso neste momento.

Insira o arquivo '**270_cabos.dat**' no lugar do texto '*nome*' da célula abaixo e execute-a para carregar o arquivo de dados. Coloque o nome do arquivo sempre entre aspas simples.

In [ ]:
arquivo = '270_cabos.dat'
dados = gpr.radargrama(arquivo, ruido=0.0e6)
dados.plota_dados(x=0.0)

Na terceira linha desta célula de código acima da figura é possível atribuir um valor numérico à variável $x$ entre parêntesis. Este valor numérico corresponde a uma posição no radargrama (figura à esquerda) indicada por uma linha tracejada preta. O traço adquirido sobre esta posição é mostrado individualmente na figura à direita, assim você pode modificar este valor numérico como quiser para observar as características dos diferentes traços que compõem o radargrama. Altere este valor e verifique as características do sinal nas posições em que há hipérboles e nas posições em que não há nenhuma. Se o valor de $x$ estiver fora dos limites do radargrama a imagem exibirá automaticamente o traço correspondente a uma das extremidades do perfil.

# Passo 03 - Aplicação de ganho aos dados

Observe que os dados carregados no **Passo 02** apresentam informações muito mais claras na sua porção superior do que na sua porção inferior. Este efeito é causado pela atenuação do sinal conforme este se propaga a maiores profundidades em função da condutividade elétrica do meio geológico. Para compensar esta atenuação é necessário aplicar um *ganho* aos dados, isto é, aplicar um fator multiplicador a trechos do sinal de forma que a amplitude registrada torne as feições de interesse mais claramente visíveis na imagem. Estes valores são aplicados a cada amostra de sinal de cada traço no eixo de tempo. Existem diversos tipos de ganho que podem ser aplicados a um sinal, porém aqui você irá ver apenas os três principais: **ganho linear**, **ganho exponencial** e **ganho AGC**.

### Ganho linear
Corresponde a uma função do tipo $a t$ estritamente crescente, assumindo o valor igual a zero no tempo mínimo, o valor máximo $a$ no tempo final do registro e valores intermediários com uma distribuição linear entre estes dois tempos.

A célula abaixo apresenta este tipo de ganho. A variável $a$ representa o fator multiplicador máximo do ganho, a variável $x$ seleciona um traço em uma posição específica (em metros) para a visualização individual, a variável $tmin$ seleciona o tempo mínimo (em segundos) a partir do qual o ganho será aplicado e a variável $salva$ serve para armazenar o ganho aplicado ao dado caso você esteja satisfeito com a imagem obtida. Por enquanto mantenha a variável $salva$ com o valor **False** e altere os valores numéricos de $a$, $tmin$ e $x$ para avaliar o efeito do ganho linear sobre a imagem do radargrama. A função de ganho aplicada será mostrada em escala normalizada sobre o traço na figura à direita. Execute a célula após definir estes valores a seu critério.

**Obs.**: a palavra *temporário* irá aparecer na figura ao lado do nome do arquivo, indicando que o ganho foi aplicado porém ainda não foi salvo sobre os dados). Assim, quaisquer valores que você inserir serão aplicados apenas nos dados brutos visualizados no **Passo 02**.

In [ ]:
dados.ganho_linear(a=10.0, tmin=0.0e-9, x=0.0, salva=False)

### Ganho exponencial

Corresponde a uma função do tipo $a e^{bt}$ estritamente crescente, assumindo o valor mínimo $a$ no tempo $t$=0.0 ns, o valor máximo $a e^{bt}$ no tempo final do registro e valores intermediários com uma distribuição exponencial entre estes dois tempos.

A célula abaixo apresenta este tipo de ganho. A variável $a$ representa o fator multiplicador linear do ganho, a variável $b$ representa o fator multiplicador exponencial do ganho, a variável $x$ seleciona um traço em uma posição específica (em metros) para a visualização individual, a variável $tmin$ seleciona o tempo mínimo (em segundos) a partir do qual o ganho será aplicado e a variável $salva$ serve para armazenar o ganho aplicado ao dado caso você esteja satisfeito com a imagem obtida. Por enquanto mantenha a variável $salva$ com o valor **False** e altere os valores numéricos de $a$, $b$, $tmin$ e $x$ para avaliar o efeito do ganho exponencial sobre a imagem do radargrama. A função de ganho aplicada será mostrada em escala normalizada sobre o traço na figura à direita. Execute a célula após definir estes valores a seu critério.

**Obs.**: a palavra *temporário* irá aparecer na figura ao lado do nome do arquivo, indicando que o ganho foi aplicado porém ainda não foi salvo sobre os dados). Assim, quaisquer valores que você inserir serão aplicados apenas nos dados brutos visualizados no **Passo 02**.

In [ ]:
dados.ganho_exp(a=1.0, b=1.0, tmin=0.0e-9, x=0.0, salva=False)

### Ganho AGC

O AGC (*Automatic Gain Control*, Controle Automático de Ganho) é o tipo de ganho mais comum aplicados a dados de GPR e a dados sísmicos. Ele tende a equalizar as amplitudes do traço e corresponde a uma função que é calculada dinamicamente em função da própria amplitude apresentada pelo sinal. Define-se uma janela de tempo (menor do que o tempo máximo de registro do sinal) dentro da qual é calculado o valor da amplitude RMS, dado por:

$$A_{RMS} = \sqrt{\frac{1}{N}\sum_{i=1}^{N}{|s_{i}|^2}}$$

em que $N$ corresponde ao número de amostras dentro da janela de tempo definida e $s_i$ corresponde à $i$-ésima amostra do sinal. A razão entre uma amplitude RMS fixa de referência $A_{REF}$ e a amplitude RMS $A_{RMS}$ é calculada e o valor resultante é adotado como fator multiplicador a ser aplicado em cada amostra desta janela. A janela de tempo é então deslocada de uma amostra no tempo e o procedimento acima é repetido até que a janela percorra todo o traço. Isto é feito para cada traço do radargrama, de forma haverá uma função de ganho característica calculada automaticamente para cada traço.

A célula abaixo apresenta este tipo de ganho. A variável $tw$ representa o intervalo da janela de tempo desejada (em segundos) a variável $salva$ serve para armazenar o ganho aplicado ao dado caso você esteja satisfeito com a imagem obtida. Por enquanto mantenha a variável $salva$ com o valor **False** e altere o valor numéricos de $tw$ para avaliar o efeito do ganho AGC sobre a imagem do radargrama. A função de ganho aplicada será mostrada em escala normalizada sobre o traço na figura à direita. Execute a célula após definir este valor a seu critério.

**Obs.**: a palavra *temporário* irá aparecer na figura ao lado do nome do arquivo, indicando que o ganho foi aplicado porém ainda não foi salvo sobre os dados). Assim, quaisquer valores que você inserir serão aplicados apenas nos dados brutos visualizados no **Passo 02**.

In [ ]:
dados.ganho_agc(tw=10.0e-9, salva=False)

Escolha o ganho que você julgar mais adequado dentre os três analisados acima e, assim que você definir valores numéricos satisfatórios, modifique o valor da variável $salva$ para **True** na célula correspondente. Isto irá armazenar as amplitudes do radargrama após o ganho ser aplicado. Observe que agora a palavra temporário não aparece mais ao lado do nome do arquivo, indicando que a alteração foi efetuada de forma definitiva. A função de ganho também não é mais apresentada na figura à direita, ficando apenas as amplitudes definitivas do traço. **Não avance para o próximo passo antes de aplicar o ganho definitivamente sobre os dados**.

**Obs. 1**: A aplicação do ganho não pode ser desfeita, então se você acidentalmente salvar um ganho inadequado precisará carregar o arquivo de dados novamente no **Passo 02** para corrigir o erro.

**Obs. 2**: Uma vez que você salva o dado com ganho ele torna-se o conjunto de dados padrão. Isto significa que se você executar alguma outra função de ganho ela será aplicada sobre os dados já amplificados, ou seja, você irá aplicar um ganho sobre outro ganho. Isto é sempre possível porém não é recomendado, pois juntamente com o dado que nos interessa há a amplificação também do ruído que pode estar presente no dado.

# Passo 04 - Filtragem

Caso o ruído do dado tenha sido amplificado (o que quase sempre acontece em um dado real de campo) é necessário removê-lo para possibilitar a visualização das feições de interesse no radargrama. Contudo, é necessário antes analisar o *espectro* do radargrama para avaliar o que pode ser ruído e o que pode não ser.

Execute a célula abaixo para visualizar o espectro médio do radargrama, que irá lhe indicar a faixa em que as frequências de maior amplitude do sinal se concentram.

In [ ]:
dados.plota_espectro()

**Sugestão**: Se você estiver com muitas dúvidas ou não se lembrar dos fundamentos da filtragem pode acessar o [notebook sobre filtragem](https://mybinder.org/v2/gh/emrodalmeida/demoaulas/HEAD?filepath=Geofisica1%2Ffiltragem.ipynb) elaborado para a disciplina de Geofísica I. Se quiser entender por quê isso funciona, pode acessar o [notebook sobre as séries de Fourier](https://mybinder.org/v2/gh/emrodalmeida/demoaulas/HEAD?filepath=Geofisica1%2Fserie_fourier.ipynb), também elaborado para essa disciplina.

Sobre este esectro é necessário excluir algumas componentes de frequência que se considera estarem associadas ao ruído e manter as frequências que se considera que estejam associadas ao sinal de interesse. Não há uma regra para isto, visto que o ruído pode contaminar o sinal em qualquer faixa de frequência. Em geral busca-se melhorar a imagem removendo o mínimo possível das frequências do espectro e dando preferência pela remoção das frequências mais distantes da frequência nominal da antena empregada na aquisição. Não há nada que impeça a exclusão de frequências próximas à frequência nominal da antena, e de fato isto muitas vezes acaba sendo necessário, porém ao se fazer isso haverá a degradação do sinal ao invés de haver uma melhoria. Assim, deve-se sempre ponderar entre o quanto é possível remover ou manter no sinal para não degradá-lo demais.

Na célula abaixo você pode deifinir um filtro passa-banda definido por duas frequências de corte $f_{c1}$ e $f_{c2}$. O passa-banda é o tipo de filtro em que a banda de frequência existente entre as frequências $f_{c1}$ e $f_{c2}$ é mantida no espectro do sinal e todas as outras frequências são removidas. Os valores abaixo ($f_{c1}$ = 10 MHz e $f_{c2}$ = 600 MHz) estão definidos para um filtro de banda larga, ou seja, que compreende uma larga faixa de frequências a serem mantidas no espectro. Filtros passa-banda de banda larga tendem a alterar muito pouco o sinal. A primeira figura lhe mostrará o espectro original com a representação do filtro que foi definido e a segunda figura lhe mostrará o espectro após a aplicação do filtro.

In [ ]:
freq_corte = [10e6, 600e6]
dados.plota_filtragem(f_c = freq_corte)

Execute a célula abaixo para visualizar o efeito desta filtragem sobre os dados. A figura resultante lhe mostrará o radargrama sem as frequências que ficaram fora da banda definida no filtro. As variáveis $x$ e $salva$ têm a mesma função apresentada nas etapas de ganho. Você pode alterar o valor da variável $x$, mas mantenha a variável $salva$ com o valor **False** por enquanto.

In [ ]:
dados.passa_banda(freq_corte, x=1.1, salva=False)

Você irá notar que o efeito surtido pelo filtro sobre os dados foi imperceptível, pois o radargrama resultante é praticamente igual ao radargrama após a etapa de ganho. Isto é consequência da filtragem com uma banda larga, como mencionado acima.

Agora volte à célula em que foram definidas as frequências de corte $f_{c1}$ e $f_{c2}$ e defina outras bandas de filtragem. Veja como o dado é modificado. Quando encontrar uma banda satisfatória mude o valor da variável $salva$ para **True**.

**Opcional**: Caso você queira é possível aplicar a etapa de ganho novamente. Para isto basta executar novamente a célula com o ganho desejado *sem carregar novamente* o arquivo de dados. Na prática, as etapas de ganho e filtragem podem ser repetidas sucessivamente quantas vezes forem necessárias para se obter uma imagem passível de interpretação, desde que se tome o cuidado de não modificar demais o dado para evitar quaisquer vieses interpretativos.

# Passo 05 - Estimativa de velocidade das ondas EM

Em dados adquiridos em configuração de afastamento comum é possível estimar a velocidade de propagação das ondas eletromagnéticas através do ajuste de uma hipérbole teórica posicionada sobre uma hipérbole qualquer identificada no radargrama, causada pela difração das ondas em alvos pontuais. A abertura da hipérbole é determinada pela velocidade de propagação das ondas no meio geológico, considerado homogêneo. Velocidades mais altas geram hipérboles mais abertas, enquanto velocidades mais baixas geram hipérboles mais fechadas.

A célula baixo permite posicionar uma hipérbole teórica sobre os dados do arquivo carregado no **Passo 02**. A variável $v$ corresponde à velocidade de propagação das ondas (em m/ns), a variável $xc$ corresponde à posição central (em metros) da hipérbole, a variável $dx$ corresponde à extensão de cada braço da hipérbole (em metros), a variável $t0$ corresponde ao tempo do ápice da hiperbole (em segundos) e a variável $r$ corresponde ao raio do alvo pontual (em metros). Modifique estes valores e tente estimar a velocidade de propagação neste perfil. Este procedimento não modifica nada no dado em si.

**Obs.:** Se você trabalhar com o arquivo '**200_SC.dat**' você não conseguirá encontrar uma hipérbole para fazer a estimativa de velocidade (como você faria para estimá-la nesta situação?). Neste caso você pode considerar a velocidade de 0,077 m/ns, que foi determinada através de outros meios.

In [ ]:
dados.ajusta_hiperbole(v=0.030, xc=5.0, dx=0.5, t0=5e-9, r=0.5)

A informação da velocidade é importante para estimar a constante dielétrica ($ \varepsilon_{r} $) do meio através do qual as ondas se propagam, fornecendo uma indicação sobre o tipo de material atravessado, e para estimar a profundidade dos alvos e feições identificados. A constante dielétrica é calculada através da equação

$$ \varepsilon_{r} = \left(\frac{c}{v}\right)^2 $$

em que $c$ é a velocidade da luz (= 0.3 m/ns) e $v$ é a velocidade de propagação (em m/ns) estimada pelo ajuste de hipérbole. Calcule o valor de $ \varepsilon_{r} $ e insira-o na célula abaixo como valor da variável $eps$ e execute a célula para converter a escala de tempo em escala de profundidade, estimando assim a profundidade a que os alvos encontram-se. A variável $x$ tem a mesma função já discutida nos passos anteriores.

In [ ]:
dados.tempo_profundidade(eps=15.0, x=21.15)

# Passo 06 - Trabalhando com dados ruidosos

Agora reinicie o notebook, carregando novamente o mesmo arquivo de dados com o qual você trabalhou até agora, porém insira um ruído na faixa de frequência centrada em 900 MHz (900e6 Hz). Esta faixa de frequência está entre as adotadas por operadoras de telefonia celular. Execute os mesmos passos que você fez até aqui e veja como o ganho e o filtro atuam sobre os dados, interferindo na sua interpretação.